In [1]:
import syft as sy
from syft import ActionObject
from syft import syft_function, syft_function_single_use
from time import sleep
import os
import psutil
import inspect

kj/filesystem-disk-unix.c++:1703: warning: PWD environment variable doesn't match current directory; pwd = /home/teo/OpenMined/PySyft


with server

In [2]:
node = sy.orchestra.launch(name="test-domain-helm2", dev_mode=True, reset=True)
                           
client = node.login(email="info@openmined.org", password="changethis")

Staging Protocol Changes...
Data Migrated to latest version !!!
Logged into <test-domain-helm2: High side Domain> as <info@openmined.org>


SyftWarning: You are using a default password. Please change the password using `[your_client].me.set_password([new_password])`.

In [3]:
res = client.register(name="a", email="aa@b.org", password="c", password_verify="c")

In [4]:
ds_client = node.login(email="aa@b.org", password="c")

Logged into <test-domain-helm2: High side Domain> as <aa@b.org>


setup: compute train-test overlap between a very large train set and a smaller test set. Small test is still to big for memory, so we split it into 54 parts. We keep 1 of those parts in memory. We dont keep the train set in memory, but read and compare with 1/54 parts line by line. Each part takes ~30 hours, but we can run 54 processes in parallel.

## Dataset

In [5]:
dataset = sy.Dataset(
    name="My dataset",
    asset_list=[
        sy.Asset(
            name="input asset",
            data=ActionObject.from_obj([1, 2, 3]),
            mock=ActionObject.from_obj([1, 1, 1]),
        ),
    ]
)

In [6]:
dataset

syft.service.dataset.dataset.CreateDataset

In [7]:
node.python_node

Domain: test-domain-helm2 - 448cb501cca248c7b9845081ed40744c - domain

Services:
ActionService, BlobStorageService, CodeHistoryService, DataSubjectMemberService, DataSubjectService, DatasetService, EnclaveService, JobService, LogService, MetadataService, MigrateStateService, NetworkService, NotificationService, PolicyService, ProjectService, QueueService, RequestService, SettingsService, UserCodeService, UserService, WorkerService

In [8]:
from syft.service.context import AuthedServiceContext
context = AuthedServiceContext(
    credentials=client.verify_key,
    node=node.python_node,
)

In [9]:
from syft.service.dataset.dataset import Asset, Dataset

In [10]:
dataset.syft_client_verify_key

In [11]:
res = client.upload_dataset(dataset)

100%|██████████| 1/1 [00:00<00:00, 12.00it/s]

Uploading: input asset
(3,)


In [12]:
for x in res:
    print(x)

('message', "Dataset uploaded to 'test-domain-helm2'. To see the datasets uploaded by a client on this node, use command `[your_client].datasets`")


In [13]:
dataset.assets[0]

```python
class CreateAsset:
  id: str = None
  name: str = "input asset"

```

In [14]:
asset = ds_client.datasets[0].assets[0]

In [15]:
@syft_function()
def process_batch(batch):
    # takes 30 hours normally
    print("process_batch")
    from time import sleep
    sleep(1)
    return batch+1

SyftSuccess: Syft function 'process_batch' successfully created. To add a code request, please create a project using `project = syft.Project(...)`, then use command `project.create_code_request`.

In [16]:
ds_client.code.submit(process_batch)

SyftSuccess: User Code Submitted

## Batch function

In [17]:
@syft_function()
def middle_batch(domain, batch):
    # takes 30 hours normally
    print("middle_batch")
    from time import sleep
    sleep(1)
    return domain.launch_job(process_batch, batch=batch)

SyftSuccess: Syft function 'middle_batch' successfully created. To add a code request, please create a project using `project = syft.Project(...)`, then use command `project.create_code_request`.

In [18]:
ds_client.code.submit(middle_batch)

SyftSuccess: User Code Submitted

## Main function

In [19]:
@syft_function_single_use(x=asset)
def process_all(domain, x):
    import numpy as np
    print("process_all")
    print(list(globals().keys()))
    print(list(locals().keys()))
    y = np.array([1,2,3])
    jobs = []
    for elem in x:
        # We inject a domain object in the scope
        batch_job = domain.launch_job(middle_batch, batch=elem)
        jobs += [batch_job]

    return jobs

SyftSuccess: Syft function 'process_all' successfully created. To add a code request, please create a project using `project = syft.Project(...)`, then use command `project.create_code_request`.

# Approve & run

In [20]:
ds_client.code.request_code_execution(process_all)
client.requests[-1].approve()

SyftError: Multiple codes detected, please use approve_nested=True

In [21]:
mock =asset.mock

In [22]:
mock

[1, 1, 1]

In [23]:
domain = sy.orchestra.launch(name="test",
                           dev_mode=True,
                           reset=True, 
                           n_consumers=3,
                           create_producer=True,
                           queue_port=22221)
                           
temp_client = node.login(email="info@openmined.org", password="changethis")

Staging Protocol Changes...
Data Migrated to latest version !!!
Logged into <test-domain-helm2: High side Domain> as <info@openmined.org>


SyftWarning: You are using a default password. Please change the password using `[your_client].me.set_password([new_password])`.

In [24]:
filtered_kwargs = {"x": [3,4]}

In [25]:
globals()["middle_batch"]

```python
class SubmitUserCode:
  id: str = None
  func_name: str = "middle_batch"
  code: str = "@syft_function()
def middle_batch(domain, batch):
    # takes 30 hours normally
    print("middle_batch")
    from time import sleep
    sleep(1)
    return domain.launch_job(process_batch, batch=batch)
"

```

In [26]:
action_kwargs = {}
for k, v in filtered_kwargs.items():
    val = ActionObject.from_obj(v)
    action_kwargs[k] = val.send(temp_client)


In [27]:
process_all(**action_kwargs)

['__name__', '__doc__', '__package__', '__loader__', '__spec__', '__file__', '__cached__', '__builtins__', 'annotations', 'ast', 'datetime', 'Enum', 'hashlib', 'inspect', 'StringIO', 'itertools', 'sys', 'time', 'traceback', 'Any', 'Callable', 'Dict', 'List', 'Optional', 'Tuple', 'Type', 'Union', 'final', 'display', 'Err', 'Self', 'NodeType', 'APIRegistry', 'NodeIdentity', 'PythonConnection', 'EnclaveMetadata', 'SyftVerifyKey', 'get_data_protocol', '_deserialize', 'serializable', '_serialize', 'PartitionKey', 'LinkedObject', 'DateTime', 'migrate', 'SYFT_OBJECT_VERSION_1', 'SYFT_OBJECT_VERSION_2', 'SyftHashableObject', 'SyftObject', 'TransformContext', 'add_node_uid_for_key', 'drop', 'generate_id', 'make_set_default', 'transform', 'UID', 'options', 'SURFACE', 'CodeMarkdown', 'as_markdown_code', 'Action', 'ActionObject', 'AuthedServiceContext', 'Asset', 'Job', 'CustomInputPolicy', 'CustomOutputPolicy', 'EmpyInputPolicy', 'ExactMatch', 'InputPolicy', 'OutputPolicy', 'Policy', 'SingleExecut

[4, 5]